In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
import secrets     

In [28]:
#load data from 2d and 3d landmarks 

#start with BBP
Data_3D = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\NewFeatures2d_BIGSMILE.csv", index_col=0)
#Data_2D = pd.read_csv(r"C:\Users\GuarinD\Documents\GitHub\Face_and_Gestures_2020\data\Features2d_BBP.csv", index_col = 0)
#Data_3D.pop('Unnamed: 0');
#Data_2D.pop('Unnamed: 0');

In [31]:
subjects = ['NF12', 'NF13', 'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21', 'NF22', 'NF23', 'NF24', 'NF26', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06', 'PD07', 'PD08']
healthy = ['NF12', 'NF13', 'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21', 'NF22', 'NF23','NF24', 'NF26']
patients = ['PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06', 'PD07', 'PD08']

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 50)]
# number of features at every split
max_features = ['auto','log2', None]

# max depth
max_depth = [int(x) for x in np.linspace(1, 20, num = 20)]
max_depth.append(None)

criterion = ['gini', 'entropy']
# create random grid
random_grid = {
'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'criterion' : criterion
}

estimator = RandomForestClassifier()

Data = Data_3D.copy(deep=True)

YPRED = []
YTEST = []
acc = []
conf_mat = []
pred_status = []

true_labels=[1,]

disease_status = []
conf_matrix_totals =np.zeros((2,2))
conf_matrix_patients = np.zeros((2,2))
for s in subjects:
    healthy = ['NF12', 'NF13', 'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21', 'NF22', 'NF23', 'NF24', 'NF26']
    patients = ['PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06', 'PD07', 'PD08']

    #get data for all subjects except 's'
    values_ = Data.loc[Data.Subject_ID != s].values    
    X = values_[:,1:18]
    y = values_[:,18]
    
    custom_index = []
    TempDF = pd.DataFrame(values_, columns = Data.columns)
    if 'NF' in s:
        idx = healthy.index(s)
        healthy.pop(idx)
        
        rand_healthy = secrets.SystemRandom().sample(healthy, 6)
        rand_healthy = np.asarray(rand_healthy).reshape(3,2)
        rand_patients = secrets.SystemRandom().sample(patients, 6)
        rand_patients = np.asarray(rand_patients).reshape(3,2)
        for hh,pp in zip(rand_healthy,rand_patients):
            val_subjects = np.concatenate((hh,pp))
            training_index = TempDF[~TempDF.Subject_ID.isin(val_subjects)].index.values.astype(int)
            val_index = TempDF[TempDF.Subject_ID.isin(val_subjects)].index.values.astype(int)
        
            custom_index.append( (training_index, val_index) )

    else:
        idx = patients.index(s)
        healthy.pop(idx)
        rand_healthy = secrets.SystemRandom().sample(healthy, 6)
        rand_healthy = np.asarray(rand_healthy).reshape(3,2)
        rand_patients = secrets.SystemRandom().sample(patients, 6)
        rand_patients = np.asarray(rand_patients).reshape(3,2)
        for hh,pp in zip(rand_healthy,rand_patients):
            val_subjects = np.concatenate((hh,pp))
            training_index = TempDF[~TempDF.Subject_ID.isin(val_subjects)].index.values.astype(int)
            val_index = TempDF[TempDF.Subject_ID.isin(val_subjects)].index.values.astype(int)
        
            custom_index.append( (training_index, val_index) )
        

#     rfc_random = RandomizedSearchCV(estimator = estimator, param_distributions = random_grid, n_iter = 100, cv =custom_index, verbose=False, random_state=0, n_jobs = -1)
#     # Fit the model
#     rfc_random.fit(X, y.astype(np.float))

    
#     #generate a model with best parameters
#     estimator = RandomForestClassifier(**rfc_random.best_params_, n_jobs=-1)
    estimator = RandomForestClassifier(n_jobs=-1)
    estimator.fit(X,y.astype(np.float))
    #get data for subject 's'
    values = Data.loc[Data.Subject_ID == s].values    
    X_test = values[:,1:18]
    y_test = values[:,18]
    
    
    
    #evalues best model for subject s
    y_pred = estimator.predict(X_test)
#     #print(best_estimator.feature_importances_)
#     print(confusion_matrix(y_test.astype(np.float),y_pred))
#     print(classification_report(y_test.astype(np.float),y_pred))
#     print(accuracy_score(y_test.astype(np.float), y_pred))
    
    YPRED.append(y_pred)
    YTEST.append(y_test.astype(np.float))
    acc.append(accuracy_score(y_test.astype(np.float), y_pred))
    
    cm = confusion_matrix(y_test.astype(np.float),y_pred)
    if len(cm)==1:
        if 'NF'in s:
            conf_m = np.zeros((2,2))
            conf_m[0,0]=cm[0][0]
        else:
            conf_m = np.zeros((2,2))
            conf_m[1,1]=cm[0][0]
    else:
        conf_m = cm      
        
        
    conf_mat.append(conf_m)
    conf_matrix_totals = conf_matrix_totals+conf_m
    
    
    disease_status.append(0 if np.mean(y_pred)<=0.5 else 1)
    
    if 'NF' in s:
        if disease_status[-1] == 0:
            pred_status.append(1)
        else:
            pred_status.append(0)            
    else:
        if disease_status[-1] == 1:
            pred_status.append(1)
        else:
            pred_status.append(0)
            
            
    if 'NF' in s:
        if disease_status[-1] == 0:
            conf_matrix_patients = conf_matrix_patients + np.array([[1,0],[0,0]])
        else:
            conf_matrix_patients = conf_matrix_patients + np.array([[0,1],[0,0]])
            
    else:
        if disease_status[-1] == 1:
            conf_matrix_patients = conf_matrix_patients + np.array([[0,0],[0,1]])
        else:
            conf_matrix_patients = conf_matrix_patients + np.array([[0,0],[1,0]])
        


print(conf_matrix_totals,conf_matrix_patients,np.mean(np.asarray(pred_status))*100)  

[[51. 17.]
 [23. 17.]] [[11.  3.]
 [ 5.  3.]] 63.63636363636363


In [15]:
for s,p in zip(subjects,disease_status):
    print(s+'-'+str(p))


NF12-0
NF13-0
NF14-0
NF15-0
NF16-0
NF17-0
NF18-0
NF19-1
NF20-0
NF21-0
NF22-0
NF23-0
NF24-0
NF26-1
PD01-1
PD02-0
PD03-0
PD04-0
PD05-1
PD06-0
PD07-1
PD08-1


In [454]:
YPRED

[array([0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.]),
 array([1., 1., 0., 1., 0., 1., 1., 1., 1., 1.]),
 array([0., 0.]),
 array([0., 1., 1., 1., 1., 0., 1., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 1., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 0., 1., 1., 1., 1., 0., 1., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([0., 1., 0., 0., 1.])]

In [342]:
conf_matrix_totals

array([[403., 139.],
       [203.,   9.]])

In [386]:
disease_status = 0 if np.sum(np.abs(y_pred-y_test))/len(y_pred)<0.5 else 1
disease_status

1

In [391]:
disease_status = 0 if np.mean(y_pred)<=0.5 else 1
disease_status

0

In [404]:
Data.loc[Data.Subject_ID != s]

,Subject_ID,Duration,max_TB,max_TB_vel,min_TB_vel,Label
31,NF13,0.219880,0.904690,0.175109,-0.156268,0.0
32,NF13,0.179902,0.723334,0.133691,-0.140568,0.0
33,NF13,0.199891,0.645300,0.175866,-0.181723,0.0
34,NF13,0.179902,0.670724,0.183860,-0.112598,0.0
35,NF13,0.179902,0.677409,0.142000,-0.147448,0.0
36,NF13,0.199892,0.673699,0.149922,-0.159912,0.0
37,NF13,0.199892,0.689873,0.150833,-0.117375,0.0
38,NF13,0.219881,0.795321,0.189360,-0.157307,0.0
39,NF13,0.199892,0.888597,0.189405,-0.143860,0.0
40,NF13,0.199892,0.730202,0.151689,-0.185065,0.0


In [414]:
time = np.arange(len(y))
len(np.diff(time)), len(time)

(102, 103)

In [424]:
y[:]

array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.